<a href="https://colab.research.google.com/github/max-ostapenko/ga4_data_import/blob/main/scripts/GCS_to_GA4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @markdown 1. Define the input variables
!pip install git+https://github.com/max-ostapenko/ga4_data_import.git -q

GCP_PROJECT_ID = 'max-ostapenko'  #@param {type: "string"}
ZONE = 'us-central1-a' #@param {type: "string"}
INSTANCE_NAME = 'sftp-server' #@param {type: "string"}
SFTP_USERNAME = 'sftp-user' #@param {type: "string"}
BUCKET_NAME = 'sftp-server-ga4' #@param {type: "string"}

In [ ]:
# @markdown 2. Authenticate Google Colab to your GCP project
# Or run `gcloud auth application-default login` in terminal if you don't use Colab
import os

if os.getenv("COLAB_RELEASE_TAG"):
    !gcloud config set project {GCP_PROJECT_ID}
    from google.colab import auth
    auth.authenticate_user()

In [3]:
#@markdown 3. Reserve static IP for your server
from ga4_data_import.compute import create_static_address

INSTANCE_IP = create_static_address(GCP_PROJECT_ID, ZONE, INSTANCE_NAME)

print(f"""Instance IP `{INSTANCE_IP}` is reserved in your project: https://console.cloud.google.com/networking/addresses/list?project={GCP_PROJECT_ID}""")

Instance IP 35.208.143.124 is reserved in your project: https://console.cloud.google.com/networking/addresses/list?project=max-ostapenko


In [8]:
#@markdown 4. Create VM Instance for SFTP server and GCS bucket. Mount a bucket as read-only on SFTP server.
from ga4_data_import.compute import create_instance
from ga4_data_import.storage import add_bucket_read_access

create_instance(
    project_id=GCP_PROJECT_ID,
    zone=ZONE,
    instance_name=INSTANCE_NAME,
    static_address=INSTANCE_IP,
    sftp_username=SFTP_USERNAME,
    bucket_name=BUCKET_NAME
)

add_bucket_read_access(GCP_PROJECT_ID, BUCKET_NAME)

print(f"""VM instance `{INSTANCE_NAME}` is created in your project: https://console.cloud.google.com/compute/instancesDetail/zones/{ZONE}/instances/{INSTANCE_NAME}?project={GCP_PROJECT_ID}""")
print(f"""Bucket `{BUCKET_NAME}` is created in your project: https://console.cloud.google.com/storage/browser/{BUCKET_NAME}?project={GCP_PROJECT_ID}""")

print(f"""\n
Now you can enter your SFTP server url to Data Import UI:
    sftp://{INSTANCE_IP}/cost_data.csv
where `cost_data.csv` - name of a file in the bucket root""")

VM instance `sftp-server` is created in your project: https://console.cloud.google.com/compute/instancesDetail/zones/us-central1-a/instances/sftp-server?project=max-ostapenko
Bucket []`sftp-server-ga4`](https://console.cloud.google.com/storage/browser/sftp-server-ga4?project=max-ostapenko) is created


Now you can enter your SFTP server url to Data Import UI:
    sftp://35.208.143.124/cost_data.csv
where `cost_data.csv` - name of a file in the bucket root


In [9]:
# @markdown 5. Paste and push Public Key from GA4 to SFTP server for authorisation
from ga4_data_import.compute import add_shh_pub_key

KEY_VALUE = "ssh-rsa A......= Google Analytics Data Import Key"  # @param {type: "string"}
add_shh_pub_key(GCP_PROJECT_ID, ZONE, INSTANCE_NAME, SFTP_USERNAME, KEY_VALUE)

print(f"""Public Key is added to your SFTP server. Now you can connect to your SFTP server with your private key.""")

Public Key is added to your SFTP server. Now you can connect to your SFTP server with your private key.


In [ ]:
#@markdown 6. Test your SFTP connention
import os
from ga4_data_import.common import read_pub_key
from ga4_data_import.compute import add_shh_pub_key

# Authorise local SSH Key on the server
ssh_key_file = os.path.expanduser("~/.ssh/id_rsa.pub")
if not os.path.isfile(ssh_key_file):
    !ssh-keygen -t rsa -b 4096 -f ~/.ssh/id_rsa -N '' -q
colab_sshpubkey = read_pub_key(ssh_key_file)
add_shh_pub_key(GCP_PROJECT_ID, ZONE, INSTANCE_NAME, SFTP_USERNAME, colab_sshpubkey + " Developer's Key")

# Connect to SFTP server
!ssh-keygen -R {INSTANCE_IP}
!sftp {SFTP_USERNAME}@{INSTANCE_IP} -y